### Import Required Libraries and Set Up Environment Variables

In [1]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [2]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

# keys are set up for use

### Access the New York Times API

In [3]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL

query_url = (
    f"{url}api-key={nyt_api_key}&begin_date={begin_date}&end_date={end_date}"
    + f'&fq={filter_query}&sort={sort}&fl={field_list}'
)


In [4]:
# Create an empty list to store the reviews
reviews_list = []

# loop through pages 0-19
for pn in range(20):
    # create query with a page number
    # API results show 10 articles at a time
    paged_query_url = (f'{query_url}&page={pn}') 
    
    # Make a "GET" request and retrieve the JSON
    reviews = requests.get(paged_query_url).json()
          
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
  
    # Try and save the reviews to the reviews_list
    try:

        # loop through the reviews["response"]["docs"] and append each review to the list
        for doc in reviews["response"]["docs"]:
             reviews_list.append(doc)    
            
        # Print the page that was just retrieved
        print('Checked page',pn)
        
        # Print the page number that had no results then break from the loop
    except:
        print('Page',pn,'did not have a result')
        break

print(f'we have collected {len(reviews_list)} reviews.')
        

Checked page 0
Checked page 1
Checked page 2
Checked page 3
Checked page 4
Checked page 5
Checked page 6
Checked page 7
Checked page 8
Checked page 9
Checked page 10
Checked page 11
Checked page 12
Checked page 13
Checked page 14
Checked page 15
Checked page 16
Checked page 17
Checked page 18
Checked page 19
we have collected 200 reviews.


In [5]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
for i in range(5):
    print(json.dumps(reviews_list[i],indent=4))

# data is json format and contains the desired fields only

{
    "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
    "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
    "source": "The New York Times",
    "headline": {
        "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
        "kicker": null,
        "content_kicker": null,
        "print_headline": "The Attachment Diaries",
        "name": null,
        "seo": null,
        "sub": null
    },
    "keywords": [
        {
            "name": "subject",
            "value": "Movies",
            "rank": 1,
            "major": "N"
        },
        {
            "name": "creative_works",
            "value": "The Attachment Diaries (Movie)",
            "rank": 2,
            "major": "N"
        },
        {
            "name": "persons",
            "value": "Diment, Valentin Javier",
            "rank": 3,
            "major": "N"
        }
    ],
  

In [6]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
reviews_df = pd.json_normalize(reviews_list)
reviews_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:54:58+0000,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None
196,https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:53:12+0000,267,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None
197,https://www.nytimes.com/2017/03/02/movies/love...,Josh Kornbluth runs afoul of the Internal Reve...,The New York Times,"[{'name': 'creative_works', 'value': 'Love & T...",2017-03-02T21:44:18+0000,246,Review: It’s All Mirth and Taxes in ‘Love & Ta...,None,None,"It’s Inevitable, Mirth and Taxes",None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None
198,https://www.nytimes.com/2017/02/16/movies/ever...,A messed-up heroine is asked to choose between...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-02-16T21:45:50+0000,256,"Review: ‘Everybody Loves Somebody,’ a Rom-Com ...",None,None,Everybody Loves Somebody,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None


In [7]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
reviews_df['title'] = reviews_df['headline.main'].apply(lambda st: st[st.find("\u2018")+1:st.find("\u2019 Review")])
reviews_df.head()
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early

# complete per instructions - title is visible at the far right of the display.
# note: the lambda function does not always succeed in pulling out the title.  For that reason, not all rows in the NYT and IMDB database will join up
# when merged.  A further enhancement here would be to modify that function to pull out the correct title more consistently.

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?’ Probably a Lo
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children


In [8]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
reviews_df['keywords'] = reviews_df['keywords'].apply(extract_keywords)
reviews_df.head()

# keywords are fixed!

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"subject: Movies;persons: Kapur, Shekhar;person...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?’ Probably a Lo
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,subject: Movies;creative_works: A Tourist's Gu...,2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children


In [9]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
titles_list = reviews_df['title'].to_list()
titles_list

# it's a list

['The Attachment Diaries',
 'What’s Love Got to Do With It?’ Probably a Lo',
 'You Can Live Forever',
 'A Tourist’s Guide to Love',
 'Other People’s Children',
 'One True Loves',
 'The Lost Weekend: A Love Story',
 'A Thousand and One',
 'Your Place or Mine',
 'Love in the Time of Fentanyl',
 'Pamela, a Love Story',
 'In From the Side',
 'After Love',
 'Alcarràs',
 'Nelly & Nadine',
 'Lady Chatterley’s Lover',
 'The Sound of Christmas',
 'The Inspection',
 'Bones and All',
 'My Policeman',
 'About Fate',
 'Waiting for Bojangles',
 'I Love My Dad',
 'A Love Song',
 'Alone Together',
 'Art of Love',
 'The Wheel',
 'Thor: Love and Thunder',
 'Both Sides of the Blade',
 'Fire of Love',
 'Love & Gelato',
 'Stay Prayed Up',
 'Benediction',
 'Dinner in America',
 'In a New York Minute',
 'Anaïs in Love',
 'I Love America',
 'See You Then',
 'La Mami',
 'Love After Love',
 'Deep Water',
 'Lucy and Desi',
 'Cyrano',
 'The In Between',
 'Book of Love',
 'Lingui, the Sacred Bonds',
 'The Pink Clo

### Access The Movie Database API

In [10]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

In [11]:
# Create an empty list to store the results
tmdb_movies_list = []

# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter = 1

# Loop through the titles
for t in titles_list:
    # Check if we need to sleep before making a request
    if request_counter % 50 == 0:
        print('sleeping...')
        time.sleep(1)
    
    # Add 1 to the request counter
    request_counter += 1
    
    # Perform a "GET" request for The Movie Database
    query_url = (f"https://api.themoviedb.org/3/search/movie?query={t.replace(' ','+')}{tmdb_key_string}")
 
    response = requests.get(query_url).json()

    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
    try:
        # Get movie id
        this_id = response['results'][0]['id']
        #print('this_id is',this_id)
        
        # Make a request for a the full movie details
        details_url = (f'https://api.themoviedb.org/3/movie/{this_id}?{tmdb_key_string}')
        
        # Execute "GET" request with url
        response_details = requests.get(details_url).json()
                 
        # Extract the genre names into a list
        genre_list = response_details['genres']
        genres = []
        for g in genre_list:
            genres.append(g['name'])
        
        # Extract the spoken_languages' English name into a list
        spoken_languages_list = response_details['spoken_languages']
        spoken_languages = []
        for l in spoken_languages_list:
            spoken_languages.append(l['english_name'])
              
        # Extract the production_countries' name into a list
        countries_list = response_details['production_countries']
        production_countries = []
        for c in countries_list:
            production_countries.append(c['name'])
        
        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list
        movie_details = {'title':response_details['title'], 'original_title':response_details['original_title'], 
                         'budget':response_details['budget'], 'original_language':response_details['original_language'],
                         'homepage':response_details['homepage'], 'overview':response_details['overview'],
                         'popularity':response_details['popularity'], 'runtime':response_details['runtime'],
                         'revenue':response_details['revenue'],'genres':genres, 'spoken_languages':spoken_languages, 
                         'production_countries':production_countries}
        
        tmdb_movies_list.append(movie_details)
        
        # Print out the title that was found
        print(f'Request {request_counter-1} found {t}')
        
    except:
        print(f'Request {request_counter-1} DID NOT find {t}')
        
# it worked - got 200 movie reviews.  But some of the titles seemed formatted poorly and did not match titles in the movie database hmmmm.

Request 1 found The Attachment Diaries
Request 2 DID NOT find What’s Love Got to Do With It?’ Probably a Lo
Request 3 found You Can Live Forever
Request 4 found A Tourist’s Guide to Love
Request 5 found Other People’s Children
Request 6 found One True Loves
Request 7 found The Lost Weekend: A Love Story
Request 8 found A Thousand and One
Request 9 found Your Place or Mine
Request 10 found Love in the Time of Fentanyl
Request 11 found Pamela, a Love Story
Request 12 found In From the Side
Request 13 found After Love
Request 14 found Alcarràs
Request 15 found Nelly & Nadine
Request 16 found Lady Chatterley’s Lover
Request 17 found The Sound of Christmas
Request 18 found The Inspection
Request 19 found Bones and All
Request 20 found My Policeman
Request 21 found About Fate
Request 22 found Waiting for Bojangles
Request 23 found I Love My Dad
Request 24 found A Love Song
Request 25 found Alone Together
Request 26 found Art of Love
Request 27 found The Wheel
Request 28 found Thor: Love and 

In [12]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
for i in range(5):
    print(json.dumps(tmdb_movies_list[i],indent=4))

# 5 records printed below

{
    "title": "The Attachment Diaries",
    "original_title": "El apego",
    "budget": 0,
    "original_language": "es",
    "homepage": "",
    "overview": "Argentina, 1970s. A desperate young woman goes to a clinic to have a clandestine abortion. As her pregnancy is already through the fourth month, the doctor refuses. Instead, she proposes to sell the baby to one of her clients and offers to provide shelter in her house until the child is born. Their disturbed personalities will become intertwined in a strange and dangerous relationship.",
    "popularity": 5.486,
    "runtime": 102,
    "revenue": 0,
    "genres": [
        "Drama",
        "Mystery",
        "Thriller",
        "Horror"
    ],
    "spoken_languages": [
        "Spanish"
    ],
    "production_countries": [
        "Argentina"
    ]
}
{
    "title": "You Can Live Forever",
    "original_title": "You Can Live Forever",
    "budget": 0,
    "original_language": "en",
    "homepage": "https://gooddeedentertainment.c

In [13]:
# Convert the results to a DataFrame
tmdb_df = pd.DataFrame(tmdb_movies_list)
tmdb_df

,title,original_title,budget,original_language,homepage,overview,popularity,runtime,revenue,genres,spoken_languages,production_countries
0,The Attachment Diaries,El apego,0,es,,"Argentina, 1970s. A desperate young woman goes...",5.486,102,0,"[Drama, Mystery, Thriller, Horror]",[Spanish],[Argentina]
1,You Can Live Forever,You Can Live Forever,0,en,https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",29.347,96,15055,"[Drama, Romance]","[English, French]","[Canada, United States of America]"
2,A Tourist's Guide to Love,A Tourist's Guide to Love,0,en,https://www.netflix.com/title/81424906,"After an unexpected break up, a travel executi...",24.422,96,0,"[Romance, Comedy]","[English, Vietnamese]",[United States of America]
3,Other People's Children,Les Enfants des autres,0,fr,https://www.wildbunch.biz/movie/other-peoples-...,"Rachel loves her life, her students, her frien...",8.306,104,84178,"[Drama, Comedy]","[English, French]",[France]
4,One True Loves,One True Loves,0,en,,Emma and Jesse are living the perfect life tog...,14.097,100,37820,"[Romance, Drama, Comedy]","[English, Spanish]","[Czech Republic, United States of America]"
...,...,...,...,...,...,...,...,...,...,...,...,...
149,The Last Face,The Last Face,0,en,,"Miguel, a heroic Spanish doctor, puts himself ...",25.462,130,0,[Drama],"[English, Portuguese]",[United States of America]
150,Lost in Paris,Paris pieds nus,0,fr,,Fiona visits Paris for the first time to assis...,11.222,83,0,[Comedy],[French],"[Belgium, France]"
151,The Other Half,The Other Half,0,en,http://theotherhalfmovie.com/,A grief-stricken man and a bipolar woman fall ...,3.997,103,0,"[Drama, Romance]",[English],[Canada]
152,The Ottoman Lieutenant,The Ottoman Lieutenant,40000000,en,https://www.uphe.com/movies/the-ottoman-lieute...,"Lillie, a determined American woman, ventures ...",15.217,111,413844,"[Romance, Drama, War]","[Turkish, English]","[Turkey, United States of America]"


### Merge and Clean the Data for Export

In [14]:
# Merge the New York Times reviews and TMDB DataFrames on title
combined_df = pd.merge(tmdb_df,reviews_df,on='title')
print(f'Original dataframes were {len(tmdb_df)} and {len(reviews_df)} rows.  Combined dataframe contains only rows with matching titles, and is {len(combined_df)} rows.')

# dataframes are combined, on title

Original dataframes were 154 and 200 rows.  Combined dataframe contains only rows with matching titles, and is 125 rows.


In [15]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
columns_to_fix = ['genres','spoken_languages','production_countries']

# Create a list of characters to remove
characters_to_remove = ["[", "]", "'"]

# Loop through the list of columns to fix
for col in columns_to_fix:

    # Convert the column to type 'str'
    combined_df[col] = combined_df[col].astype(str)

    for x in characters_to_remove:
        combined_df[col] = combined_df[col].str.replace(x,'')

# Display the fixed DataFrame
combined_df[columns_to_fix].head()

# printing just the changed columns - the characters are removed as intended

,genres,spoken_languages,production_countries
0,"Drama, Mystery, Thriller, Horror",Spanish,Argentina
1,"Drama, Romance","English, French","Canada, United States of America"
2,"Romance, Drama, Comedy","English, Spanish","Czech Republic, United States of America"
3,Documentary,English,United States of America
4,"Drama, Crime","English, Portuguese, Spanish",United States of America


In [16]:
# Drop "byline.person" column
combined_df.drop('byline.person',axis=1,inplace=True)
combined_df.columns

# column is dropped - byline.person no longer appears in the columns list

Index(['title', 'original_title', 'budget', 'original_language', 'homepage',
       'overview', 'popularity', 'runtime', 'revenue', 'genres',
       'spoken_languages', 'production_countries', 'web_url', 'snippet',
       'source', 'keywords', 'pub_date', 'word_count', 'headline.main',
       'headline.kicker', 'headline.content_kicker', 'headline.print_headline',
       'headline.name', 'headline.seo', 'headline.sub', 'byline.original',
       'byline.organization'],
      dtype='object')

In [17]:
# Delete duplicate rows and reset index
combined_df = combined_df.drop_duplicates()
combined_df.reset_index(inplace=True)
len(combined_df)

# Duplicate rows, if there had been any, are now gone, and we retain 125 rows in our combined data frame.

125

In [18]:
# Export data to CSV without the index
combined_df.to_csv('moviesandreviews.csv',index=False)

# output is created.  NOTE that there are more columns in the "example.csv" file than in this file only because instructions for the challenge
# limited our dataframe to certain columns.  Those columns were not the same ones as in the example output file.  

# The exported file looks good and is easily read in excel.